In [1]:
!pip install fastapi uvicorn wandb pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 336.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.1 MB/s eta 0:00:00


In [2]:
!ngrok authtoken 2q1ziptsvffdmYtFNqT03RBVg7M_2WBXBCe68XPkF12xJyNN2

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
import os

In [4]:
os.environ["WANDB_API_KEY"] = "e3c34be454efd9eb6b608e015cebcc602e8dbc09"

In [48]:
# Write the FastAPI app programmatically
app_code = """
import os
import wandb
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from joblib import load
import pandas as pd
import numpy as np

app = FastAPI()

# Define input data schema
# Define input data schema
class PredictionRequest(BaseModel):
    gender: str
    race_ethnicity: str  # Updated column name
    parental_level_of_education: str
    lunch: str
    test_preparation_course: str
    reading_score: float
    writing_score: float


# Load the model from WandB
def load_model_from_wandb(project_name: str, artifact_name: str, model_file: str):
    try:
        # Initialize WandB
        run = wandb.init(project=project_name, job_type="inference", reinit=True)
        artifact = run.use_artifact(artifact_name)
        artifact_dir = artifact.download()
        model_path = os.path.join(artifact_dir, model_file)
        model = load(model_path)
        run.finish()
        return model
    except Exception as e:
        raise RuntimeError(f"Failed to load model: {e}")

project_name = "mlops_student_performance_assignment_CT1"
artifact_name = "Linear_Model_StudentsPerformance:latest"
model_file = "students_linear_model.pkl"
ml_model = load_model_from_wandb(project_name, artifact_name, model_file)

@app.post("/predict")
def predict(input_data: PredictionRequest):
    try:
        input_dict = input_data.dict()
        df = pd.DataFrame([input_dict])
        prediction = ml_model.predict(df)
        return {"Predicted Math Score": np.round(prediction[0], 2)}
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Prediction error: {e}")
"""

# Save the FastAPI app code to a file
with open("app.py", "w") as f:
    f.write(app_code)



Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [49]:
!nohup uvicorn app:app --host 0.0.0.0 --port 9090 &

nohup: appending output to 'nohup.out'


In [50]:
!ps -ax | grep uvicorn

  16144 ?        Sl     0:02 /usr/bin/python3 /usr/local/bin/uvicorn app:app --host 0.0.0.0 --port 9
  16178 ?        S      0:00 /bin/bash -c ps -ax | grep uvicorn
  16180 ?        R      0:00 grep uvicorn


In [52]:
!tail nohup.out

wandb:   1 of 1 files downloaded.  
wandb:                                                                                
wandb: 🚀 View run fragrant-cloud-8 at: https://wandb.ai/priyesh-jagtap91-tatatele/mlops_student_performance_assignment_CT1/runs/1x0ept6s
wandb: ⭐️ View project at: https://wandb.ai/priyesh-jagtap91-tatatele/mlops_student_performance_assignment_CT1
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20241214_083725-1x0ept6s/logs
INFO:     Started server process [16144]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:9090 (Press CTRL+C to quit)


In [51]:
from pyngrok import ngrok

# Expose the FastAPI app
public_url = ngrok.connect(9090)
print(f"Public URL: {public_url}")

Public URL: NgrokTunnel: "https://43ba-35-237-216-244.ngrok-free.app" -> "http://localhost:9090"


In [56]:
# Test the Prediction Service
import requests
import json

data = {
    "gender": "female",
    "race_ethnicity": "group C",  # Updated column name
    "parental_level_of_education": "some college",
    "lunch": "standard",
    "test_preparation_course": "completed",
    "reading_score": 90,
    "writing_score": 88
}





In [57]:
response = requests.post(f"https://43ba-35-237-216-244.ngrok-free.app/predict", json=json_data)
print(f"Prediction Response: {response.json()}")


Prediction Response: {'Predicted Math Score': 66.09}
